In [22]:
import pandas as pd
from rapidfuzz import fuzz, process

In [23]:
decisions = pd.read_csv('decisions_df.csv', encoding='cp1252')

In [24]:
decisions.columns

Index(['gvkey', 'Name', 'sector_name', 'carbon_major', 'date', 'case',
       'impact', 'country', 'region', 'Positive or negative', 'Column1',
       'Column2'],
      dtype='object')

In [25]:
#remove column1 and column 2
decisions = decisions.drop(columns=['Column1', 'Column2'])

In [26]:
decisions.isnull().sum()


gvkey                   150
Name                      0
sector_name             149
carbon_major            150
date                      1
case                      1
impact                  150
country                 150
region                  150
Positive or negative    108
dtype: int64

In [27]:
decisions = decisions.drop(columns = ['gvkey', 'sector_name', 'carbon_major', 'country', 'region'])

In [28]:
decisions

,Name,date,case,impact,Positive or negative
0,Williams Companies,9/30/2021,Adorers of the Blood of Christ v. Transcontine...,3.0,NaN
1,Royal Dutch Shell,6/8/2020,Advertising Standards Authority's Ruling on Sh...,-3.0,NaN
2,Ryanair,2/5/2020,ASA Ruling on Ryanair Ltd t/a Ryanair Ltd,-3.0,NaN
3,De Longhi Spa,4/29/2008,Australian Competition & Consumer Commission v...,-3.0,NaN
4,General Motors Co,9/18/2008,Australian Competition & Consumer Commission v...,-3.0,NaN
...,...,...,...,...,...
252,Volkswagen AG,11/16/2023,Altroconsumo v. Volkswagen Aktiengesellschaft ...,NaN,Negative
253,Bayerische Motoren Werke AG,2/7/2024,ASA Ruling on BMW (UK) Ltd,NaN,Negative
254,Air France KLM,12/6/2023,ASA Ruling on Air France KLM,NaN,Negative
255,Deutsche Lufthansa AG,12/6/2023,ASA Ruling on Lufthansa (complaint no.2 in 2023),NaN,Negative


In [29]:
filings = pd.read_csv('original_df.csv', encoding='cp1252')

In [30]:
filings.columns

Index(['gvkey', 'Name', 'sector_name', 'carbon_major', 'date', 'case',
       'country', 'region', 'novel', 'courtlaw', 'govplaintiff', 'interest',
       'damages', 'major_changes', 'novel_weight', 'courtlaw_weight',
       'govplaintiff_weight', 'interest_weight', 'damages_weight',
       'major_weight', 'profile_sum', 'Unnamed: 21', '2004-01-01',
       '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01',
       '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01',
       '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01',
       '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01',
       '2025-01-01', 'Unnamed: 44', 'name', 'country.1', 'region.1',
       'gic_sector', 'Total # filings', 'Unnamed: 50', '2004-01-01.1',
       '2005-01-01.1', '2006-01-01.1', '2007-01-01.1', '2008-01-01.1',
       '2009-01-01.1', '2010-01-01.1', '2011-01-01.1', '2012-01-01.1',
       '2013-01-01.1', '2014-01-01.1', '2015-01-01.1', '20

In [31]:
columns_to_keep = ['Name', 'date', 'case']
filings = filings[columns_to_keep]

In [32]:
#remove rows after 397 
filings = filings.iloc[:396]

In [33]:
#rename date column to 'date_filing'
filings = filings.rename(columns={'date': 'date_filing'})

decisions = decisions.rename(columns={'date': 'date_decision'})

In [34]:
decisions

,Name,date_decision,case,impact,Positive or negative
0,Williams Companies,9/30/2021,Adorers of the Blood of Christ v. Transcontine...,3.0,NaN
1,Royal Dutch Shell,6/8/2020,Advertising Standards Authority's Ruling on Sh...,-3.0,NaN
2,Ryanair,2/5/2020,ASA Ruling on Ryanair Ltd t/a Ryanair Ltd,-3.0,NaN
3,De Longhi Spa,4/29/2008,Australian Competition & Consumer Commission v...,-3.0,NaN
4,General Motors Co,9/18/2008,Australian Competition & Consumer Commission v...,-3.0,NaN
...,...,...,...,...,...
252,Volkswagen AG,11/16/2023,Altroconsumo v. Volkswagen Aktiengesellschaft ...,NaN,Negative
253,Bayerische Motoren Werke AG,2/7/2024,ASA Ruling on BMW (UK) Ltd,NaN,Negative
254,Air France KLM,12/6/2023,ASA Ruling on Air France KLM,NaN,Negative
255,Deutsche Lufthansa AG,12/6/2023,ASA Ruling on Lufthansa (complaint no.2 in 2023),NaN,Negative


In [35]:
filings

,Name,date_filing,case
0,American Electric Power,7/21/2004,Connecticut v. Am. Elec. Power
1,General Motors Co,9/20/2006,California v. GM Corp.
2,Volkswagen,5/7/2007,Germanwatch vs. Volkswagen
3,General Motors Co,1/16/2008,Australian Competition & Consumer Commission v...
4,Duke Energy,2/12/2008,Sierra Club v. Duke Energy Indiana
...,...,...,...
391,Delta Air Lines,11/30/2023,Complaint against Virgin Atlantic and British ...
392,Repsol SA,2/21/2024,Iberdrola and others vs. Repsol
393,ING Group,1/19/2024,Milieudefensie v. ING Bank
394,AIG,8/10/2022,Aloha Petroleum Ltd. v. National Union Fire In...


In [36]:
import re
import unicodedata

def clean_text(text, 
               lowercase=True, 
               remove_punctuation=False, 
               remove_numbers=False, 
               strip_non_ascii=True, 
               normalize_unicode=True):
    """Standardized text cleaning function with configurable options."""
    
    if not isinstance(text, str):
        return text  # Return as-is if it's not a string

    if normalize_unicode:
        text = unicodedata.normalize('NFKC', text)
    
    if lowercase:
        text = text.lower()

    if strip_non_ascii:
        text = text.encode('ascii', 'ignore').decode('ascii')
        
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)

    if remove_numbers:
        text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text)  # Collapse multiple spaces
    text = text.strip()               # Remove leading/trailing spaces
    
    return text

In [37]:
filings['cleaned_name'] = filings['case'].apply(clean_text,
    lowercase=True, 
    remove_punctuation=True, 
    remove_numbers=True, 
    strip_non_ascii=True, 
    normalize_unicode=True
)
decisions['cleaned_name'] = decisions['case'].apply(clean_text,
    lowercase=True, 
    remove_punctuation=True, 
    remove_numbers=True, 
    strip_non_ascii=True, 
    normalize_unicode=True
)

In [38]:
filings['cleaned_full'] = filings['cleaned_name'] + " " + filings['Name'].fillna('').apply(clean_text)
decisions['cleaned_full'] = decisions['cleaned_name'] + " " + decisions['Name'].fillna('').apply(clean_text)

In [39]:
def fuzzy_merge(df1, df2, key1, key2, threshold=80):
    s = df2[key2].tolist()

    matches = df1[key1].apply(
        lambda x: process.extractOne(x, s, scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
    )

    df1['best_match'] = matches.apply(lambda x: x[0] if x else None)
    df1['match_score'] = matches.apply(lambda x: x[1] if x else None)

    # Debug non-perfect matches
    print("Sample non-perfect matches:")
    display(df1[df1['match_score'] < 100][['case', 'best_match', 'match_score']])

    # Merge based on best match
    merged = df1.merge(df2, left_on='best_match', right_on=key2, how='left')
    
    return merged

In [40]:
merged = fuzzy_merge(
    df1=filings.copy(),
    df2=decisions.copy(),
    key1='cleaned_full',
    key2='cleaned_full',
    threshold=85
)


Sample non-perfect matches:


,case,best_match,match_score
105,City of New York v. BP p.l.c.,city of new york v bp plc bp,85.245902
296,Kaiser et al v. Volkswagen AG,kaiser et al v volkswagen ag volkswagen ag,96.296296
319,Berrin v. Delta Air Lines Inc.,berrin v delta air lines inc delta air lines inc,95.652174
320,"Long v. Koninklijke Luchtvaart Maatschappij, N.V.",long v koninklijke luchtvaart maatschappij nv ...,97.560976
323,Simijanovic v. Koninklijke Luchtvaart Maatscha...,dakus v koninklijke luchtvaart maatschappij nv...,86.567164
332,ASA Ruling on Shell UK Ltd (following a compla...,asa ruling on shell uk ltd following a complai...,92.405063
345,Sierra Club Canada Foundation et al. v. Minist...,sierra club canada foundation et al v minister...,98.058252
347,Allhoff Cramer v. Volkswagen AG,allhoff cramer v volkswagen ag volkswagen ag,96.470588
349,Church of England Pensions Board and others v....,church of england pensions board and others v ...,97.902098


In [41]:
columns_to_drop =['cleaned_name_x', 'cleaned_full_x', 'best_match', 'match_score', 'cleaned_name_y', 'cleaned_full_y']
merged = merged.drop(columns=columns_to_drop)

In [42]:
column_order = ['Name_x', 'Name_y', 'case_x', 'case_y', 'date_filing', 'date_decision', 'impact', 'Positive or negative']

merged = merged[column_order]

In [43]:
merged

,Name_x,Name_y,case_x,case_y,date_filing,date_decision,impact,Positive or negative
0,American Electric Power,American Electric Power,Connecticut v. Am. Elec. Power,Connecticut v. Am. Elec. Power,7/21/2004,6/20/2011,3.0,NaN
1,American Electric Power,American Electric Power,Connecticut v. Am. Elec. Power,Connecticut v. Am. Elec. Power,7/21/2004,5/13/2010,3.0,NaN
2,General Motors Co,General Motors Co,California v. GM Corp.,California v. GM Corp.,9/20/2006,6/24/2009,3.0,NaN
3,General Motors Co,General Motors Co,California v. GM Corp.,California v. GM Corp.,9/20/2006,9/17/2007,3.0,NaN
4,Volkswagen,Volkswagen,Germanwatch vs. Volkswagen,Germanwatch vs. Volkswagen,5/7/2007,11/20/2007,3.0,NaN
...,...,...,...,...,...,...,...,...
446,Delta Air Lines,0,Complaint against Virgin Atlantic and British ...,NaN,11/30/2023,NaN,NaN,NaN
447,Repsol SA,Repsol SA,Iberdrola and others vs. Repsol,Iberdrola and others vs. Repsol,2/21/2024,4/16/2024,NaN,Positive
448,ING Group,0,Milieudefensie v. ING Bank,NaN,1/19/2024,NaN,NaN,NaN
449,AIG,AIG,Aloha Petroleum Ltd. v. National Union Fire In...,Aloha Petroleum Ltd. v. National Union Fire In...,8/10/2022,10/7/2024,NaN,Positive


In [44]:
#save to csv 
merged.to_csv('merged_decisions.csv', index=False, encoding='utf-8-sig')

In [47]:
merged

,Name_x,Name_y,case_x,case_y,date_filing,date_decision,impact,Positive or negative
0,American Electric Power,American Electric Power,Connecticut v. Am. Elec. Power,Connecticut v. Am. Elec. Power,7/21/2004,6/20/2011,3.0,NaN
1,American Electric Power,American Electric Power,Connecticut v. Am. Elec. Power,Connecticut v. Am. Elec. Power,7/21/2004,5/13/2010,3.0,NaN
2,General Motors Co,General Motors Co,California v. GM Corp.,California v. GM Corp.,9/20/2006,6/24/2009,3.0,NaN
3,General Motors Co,General Motors Co,California v. GM Corp.,California v. GM Corp.,9/20/2006,9/17/2007,3.0,NaN
4,Volkswagen,Volkswagen,Germanwatch vs. Volkswagen,Germanwatch vs. Volkswagen,5/7/2007,11/20/2007,3.0,NaN
...,...,...,...,...,...,...,...,...
446,Delta Air Lines,0,Complaint against Virgin Atlantic and British ...,NaN,11/30/2023,NaN,NaN,NaN
447,Repsol SA,Repsol SA,Iberdrola and others vs. Repsol,Iberdrola and others vs. Repsol,2/21/2024,4/16/2024,NaN,Positive
448,ING Group,0,Milieudefensie v. ING Bank,NaN,1/19/2024,NaN,NaN,NaN
449,AIG,AIG,Aloha Petroleum Ltd. v. National Union Fire In...,Aloha Petroleum Ltd. v. National Union Fire In...,8/10/2022,10/7/2024,NaN,Positive


In [49]:
for name in merged.Name_x.unique():
    print(name)

American Electric Power
General Motors Co
Volkswagen
Duke Energy
BP
Chevron
ExxonMobil
Royal Dutch Shell
Peabody
Dominion Energy
United Airlines Holdings Inc
Delta Air Lines
Murphy Oil Corp
Honeywell International
Equinor
Walmart Inc
Flughafen Wien
Global Partners
Costco Wholesale Corp
US Steel
Arch Resources
Hess Corp
APA Corp
Marathon Oil
Occidental 
Conocophillips
Ovintiv Inc
Anadarko
Husky Energy
Devon Energy Corp
Suncor Energy
Repsol SA
Anglo America
Rio Tinto
Total Energies SE
Eni Spa
Heidelberg Cement
RWE
CNX Resources
Glencore
Sempra Energy
ING Group
Marathon Petroleum
Arkema
Phillips 66
Consol Energy
Powszechny Zak?ad Ubezpiecze?
Keurig Dr Pepper Inc.
Enea SA
Edison International
TransDigm Group
Innospec
PG&E
PGE SA
Energy Transfers
Ascent Resources
NorthWestern Corp
Bloom Energy Corp
Williams Companies
Groupe Casino
The Coca-Cola Company
The Clorox Co
Proctor & Gamble
Nestle
Campbell Soup Company
L'Oreal
Colgate-Palmolive Co
Danimer Scientific
Oatly
Sunoco LP
HSBC
Entergy Cor